In [1]:
import torch
import config
from utils import (
    get_model_instance_segmentation,
    collate_fn,
    get_transform,
    myOwnDataset,
)
import model



In [2]:

print("Torch version:", torch.__version__)

# create own Dataset
my_dataset = myOwnDataset(
    root=config.train_data_dir, annotation=config.train_coco, transforms=get_transform()
)

# own DataLoader
data_loader = torch.utils.data.DataLoader(
    my_dataset,
    batch_size=config.train_batch_size,
    shuffle=config.train_shuffle_dl,
    num_workers=config.num_workers_dl,
    collate_fn=collate_fn,
)


# select device (whether GPU or CPU)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# DataLoader is iterable over Dataset
for imgs, annotations in data_loader:
    imgs = list(img.to(device) for img in imgs)
    annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
    print(annotations)


model = model(device = device, model_name='v2')


# parameters
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(
    params, lr=config.lr, momentum=config.momentum, weight_decay=config.weight_decay
)

len_dataloader = len(data_loader)


Torch version: 1.13.1+cpu
loading annotations into memory...


FileNotFoundError: [Errno 2] No such file or directory: 'dataset1/labels.json'

In [19]:

# Training
for epoch in range(config.num_epochs):
    print(f"Epoch: {epoch}/{config.num_epochs}")
    model.train()
    i = 0
    
    for imgs, annotations in data_loader:
        i += 1
        imgs = list(img.to(device) for img in imgs)
        annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
        print("imgs:", imgs)
        print("num: ")
        print(annotations[0]['boxes'].size() == torch.Size([0]))
        # print(annotations[1]['boxes'].size())
        # print(annotations[2]['boxes'].size())

        #if the annotations are emptpy, skip
        if annotations[0]['boxes'].size() == torch.Size([0]):
            continue
        
        print("annotations:", annotations)
        # pass if there are no annotations
        # if len(annotations) == 0:
        #     continue
        # if 
        loss_dict = model(imgs, annotations)
        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        print(f"Iteration: {i}/{len_dataloader}, Loss: {losses}")

Epoch: 0/10
imgs: [tensor([[[0.1882, 0.2118, 0.2196,  ..., 0.1882, 0.1922, 0.2078],
         [0.1725, 0.1765, 0.1686,  ..., 0.1882, 0.1804, 0.1725],
         [0.2118, 0.2000, 0.1961,  ..., 0.1529, 0.1569, 0.1490],
         ...,
         [0.7176, 0.7176, 0.7176,  ..., 0.9216, 0.9216, 0.9216],
         [0.7216, 0.7216, 0.7216,  ..., 0.9176, 0.9176, 0.9176],
         [0.7216, 0.7216, 0.7216,  ..., 0.9176, 0.9176, 0.9176]],

        [[0.2863, 0.3098, 0.3176,  ..., 0.2706, 0.2745, 0.2902],
         [0.2745, 0.2784, 0.2706,  ..., 0.2941, 0.2824, 0.2784],
         [0.3333, 0.3216, 0.3176,  ..., 0.3020, 0.2980, 0.2980],
         ...,
         [0.0039, 0.0039, 0.0039,  ..., 0.9059, 0.9059, 0.9059],
         [0.0078, 0.0078, 0.0078,  ..., 0.9020, 0.9020, 0.9020],
         [0.0078, 0.0078, 0.0078,  ..., 0.9020, 0.9020, 0.9020]],

        [[0.2745, 0.2980, 0.3059,  ..., 0.3451, 0.3490, 0.3647],
         [0.2706, 0.2745, 0.2667,  ..., 0.3608, 0.3490, 0.3451],
         [0.3451, 0.3333, 0.3294,  ...,

KeyboardInterrupt: 